# Module 2: Atomistic Machine Learning
## Part 1: Invariance

In this notebook, we illustrate the notion of invariance of descriptors with respect to operations in SO(3)

### 1. Install and load python libraries

In [ ]:
!pip install ase dscribe numpy

In [ ]:
import numpy as np
from ase import Atoms
from ase.build import molecule
from ase.visualize import view
from dscribe.descriptors import CoulombMatrix

### 2. Create molecule
Choose a molecule

In [ ]:
# Create molecule, for instance (CH4, CH3CH2NH2, H2COH ...)
mymolecule = molecule("CH4")
Natoms=len(mymolecule)
view(mymolecule, viewer='x3d')

### 3. Define transformations

In [ ]:
# Function to apply transformations
def rotate_atoms(atoms, angle=90, axis='z'):
    rotated = atoms.copy()
    rotated.rotate(a=angle, v=axis, rotate_cell=False)
    return rotated

def translate_atoms(atoms, vector):
    translated = atoms.copy()
    translated.translate(vector)
    return translated

def permute_atoms(atoms, permutation):
    positions = atoms.get_positions()[permutation]
    symbols = np.array(atoms.get_chemical_symbols())[permutation]
    return Atoms(symbols=symbols, positions=positions)

mypermutation=np.random.permutation(Natoms)

### 4. Compare atomic coordinates before and after transformations

In [ ]:
# Raw atomic coordinates invariance checks
coords_original = mymolecule.get_positions()

rotated = rotate_atoms(mymolecule, 90, 'z')
translated = translate_atoms(mymolecule, [5, 5, 5])
permuted = permute_atoms(mymolecule, mypermutation)  # arbitrary valid permutation

# Compare
np.set_printoptions(precision=6, suppress=True, floatmode='fixed')
print("Original Raw coordinates:")
print(coords_original)
print("Rotated raw coordinates")
print(rotated.get_positions())
print("Translated raw coordinates")
print(translated.get_positions())
print("Permuted raw coordinates")
print(permuted.get_positions())

- Are raw coordinates useful to characterize the atomistic configuration?

### 4. Compare Coulomb matrices before and after transformations
Coulomb matrices are defined as,
$$
C_{ij} = 
\begin{cases}
0.5 Z_i^{2.4} & \text{if } i = j \\
\frac{Z_i Z_j}{\| \mathbf{R}_i - \mathbf{R}_j \|} & \text{if } i \neq j
\end{cases}
$$
where $Z_i$ and $Z_j$ are the atomic numbers, and $R_i$ and $R_j$ the atomic coordinates of atoms $i$ and $j$, respectively.
More explicitly,

$$
\mathbf{C} =
\begin{bmatrix}
0.5 Z_1^{2.4} & \frac{Z_1 Z_2}{\| \mathbf{R}_1 - \mathbf{R}_2 \|} & \cdots \\
\frac{Z_2 Z_1}{\| \mathbf{R}_2 - \mathbf{R}_1 \|} & 0.5 Z_2^{2.4} & \cdots \\
\vdots & \vdots & \ddots
\end{bmatrix}
$$

In [ ]:
# Coulomb matrix descriptor (sorted by norm for invariance)
cm_desc = CoulombMatrix(n_atoms_max=Natoms, permutation="sorted_l2")

# Coulomb matrix calculations
cm_original = cm_desc.create(mymolecule).reshape(Natoms,Natoms)
cm_rotated = cm_desc.create(rotated).reshape(Natoms,Natoms)
cm_translated = cm_desc.create(translated).reshape(Natoms,Natoms)
cm_permuted = cm_desc.create(permuted).reshape(Natoms,Natoms)

# Compare
print("Original Coulomb matrix (sorted):")
print(cm_original)
print("Rotated Coulomb matrix")
print(cm_rotated)
print("Translated Coulomb matrix")
print(cm_translated)
print("Permuted Coulomb matrix")
print(cm_permuted)

### 5. Questions and discussion
- Are Coulomb matrices useful to characterize the atomistic configuration?
- What happens if you change:
```
cm_desc = CoulombMatrix(n_atoms_max=Natoms, permutation="sorted_l2")
```
by,
```
cm_desc = CoulombMatrix(n_atoms_max=Natoms, permutation="random",sigma=100)
```
Why?